In [ ]:
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime
import os

CLIENT_ID = XXXX
CLIENT_SECRET = XXXX

In [ ]:
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

In [ ]:
# Pull down the sleep data for a specified date range. 
# Note that the FB API only allows up to 100 requests per hour!

start_time = pd.datetime(year = 2020, month = 2, day = 1)
end_time = pd.datetime(year = 2021, month = 4, day = 30)
all_dates = pd.date_range(start=start_time, end = end_time)

date = []
duration = []
start_time = []
total_sleep_mins = []
deep_mins = []
light_mins = []
wake_mins = []
rem_mins = []

for d in all_dates:
    
    d = d.date().strftime("%Y-%m-%d")
    day_data = auth2_client.sleep(date=d)

    if len(day_data['sleep']) == 0: 
        continue
    elif not 'stages' in day_data['summary']:
        continue

    date.append(day_data['sleep'][0]['dateOfSleep'])
    start_time.append(
        datetime.datetime.strptime(day_data['sleep'][0]['startTime'], '%Y-%m-%dT%H:%M:%S.%f').time())
    total_sleep_mins.append(day_data['sleep'][0]['minutesAsleep'])
    deep_mins.append(day_data['summary']['stages']['deep'])
    light_mins.append(day_data['summary']['stages']['light'])
    wake_mins.append(day_data['summary']['stages']['wake'])
    rem_mins.append(day_data['summary']['stages']['rem'])

sleep_data = pd.DataFrame(
    {
        'date': date, 
        'start_time': start_time,
        'total_sleep_mins': total_sleep_mins,
        'deep_mins': deep_mins,
        'light_mins': light_mins,
        'wake_mins': wake_mins,
        'rem_mins': rem_mins
    })


In [ ]:
summary_df.to_csv('data/sleep.csv', index=False)

In [ ]:
sleep_data = pd.read_csv('data/sleep.csv')
sleep_data.drop(columns=['efficiency'], inplace=True)

In [ ]:
# I've been hand-recording some daily metrics, in particular my general mood and how well I felt I had slept
general_data_source = pd.read_csv('data/daylio_export_2021_05_23.csv')

def get_sleep_status(x):
    sleep_status = 'unknown'
    if "good sleep" in x:
        sleep_status = 'good'
    elif "medium sleep" in x:
        sleep_status = 'medium'
    elif "bad sleep" in x:
        sleep_status = 'bad'
    return sleep_status

general_data = pd.DataFrame(
    {
        'date': general_data_source.full_date,
        'sleep': general_data_source.activities.apply(lambda x: get_sleep_status(str(x))),
        'mood': general_data_source.mood
    }
)


In [ ]:
sleep_data = sleep_data.merge(general_data, on='date')

In [ ]:
pd.crosstab(sleep_data.sleep, sleep_data.mood)

In [ ]:
# Create some derived features

# More than 7 hours sleep
sleep_data['enough_sleep'] = sleep_data.total_sleep_mins > (7 * 60)
# More than 45 mins deep sleep
sleep_data['enough_deep_sleep'] = sleep_data.deep_mins > (45)
# Awoken in the night (pesky kids!)
sleep_data['rudely_awoken'] = sleep_data.wake_mins > (30)


In [ ]:
pd.crosstab(sleep_data.sleep, sleep_data.enough_sleep)


In [ ]:
pd.crosstab(sleep_data.sleep, sleep_data.enough_deep_sleep)

In [ ]:
pd.crosstab(sleep_data.sleep, sleep_data.rudely_awoken)

In [ ]:
# I've been hand recording my alcohol in-take
drinks_data = pd.read_csv('data/less.csv')
drinks_data = drinks_data.rename(columns = {'Date': 'date', ' Drink Count ': 'drinks'})
# The previous days consumption is the relevant data for sleep
drinks_data['date'] = drinks_data['date'].apply(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d') )

In [ ]:
sleep_data = sleep_data.merge(drinks_data, on='date')

In [ ]:
def get_drink_cat(x):
    status = 'no_drinks'
    if x > 0 and x <= 2:
        status = 'below_drinks_limit'
    elif x > 3:
        status = 'above_drinks_limit'
    return status

sleep_data['drink_status'] = sleep_data.drinks.apply(lambda x: get_drink_cat(x))

In [ ]:
pd.crosstab(sleep_data.sleep, sleep_data.drink_status)

In [ ]:
def get_bedtime_cat(x):
    if x > datetime.time.fromisoformat('12:00:00') and x <= datetime.time.fromisoformat('22:30:00'):
        status = 'early_to_bed'
    elif x > datetime.time.fromisoformat('22:30:00') and x <= datetime.time.fromisoformat('23:59:00'):
        status = 'ontime_to_bed'
    else:
        status = 'late_to_bed'
    return status

sleep_data['bedtime_status'] = sleep_data.start_time.apply(lambda x: get_bedtime_cat(datetime.time.fromisoformat(str(x))))

In [ ]:
pd.crosstab(sleep_data.sleep, sleep_data.bedtime_status)

In [ ]:
sleep_data['good_sleep'] = sleep_data.sleep == 'good'

In [ ]:
sleep_data.drop(columns=['date', 'start_time', 'total_sleep_mins', 'deep_mins', 'light_mins', 'wake_mins', 'rem_mins', 'sleep', 'mood', 'drinks'], inplace=True)

In [ ]:
bedtime_status_dummy = pd.get_dummies(sleep_data['bedtime_status'], drop_first=True)
drink_status_dummy = pd.get_dummies(sleep_data['drink_status'], drop_first=True)

In [ ]:
sleep_data = pd.concat([sleep_data, bedtime_status_dummy, drink_status_dummy], axis=1)

In [ ]:
sleep_data.drop(['bedtime_status', 'drink_status'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sleep_data.drop('good_sleep',axis=1), 
                                                    sleep_data['good_sleep'], test_size=0.30, 
                                                    random_state=99)

In [ ]:
import sklearn
from sklearn.metrics import classification_report

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2')
result = model.fit(X_train,y_train)
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(class_weight="balanced")
result = model.fit(X_train,y_train)
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1)
result = model.fit(X_train,y_train)
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))